<a href="https://colab.research.google.com/github/naikasann/Keras_tutorial/blob/master/NueralNetwork_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kerasを触ってみる (第１章 MNIST問題)

Kerasは深層学習の有名ライブラリであるTensorflowを比較的簡単に動作するように作成されたラッパーライブラリです。
これを使用することで直感的にNN(Nueral Network)を体験することができます。

では体験していきましょう

---

## Kerasを実際に触ってみる

Kerasを実際に触っていきたいと思います。
深層学習を最初に始めるうえで一番初めやすい問題がMNIST問題という問題です。
MNISTは0～9までの手書き文字画像をそれぞれどの数字なのか判別するという問題です。
まずは実際にMNISTの画像を見てみましょう。


In [ ]:
# 各種使用するライブラリをインポートします。
import tensorflow
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import plot_model

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

それぞれのライブラリについて説明していきます。
1. **Tensorflow**

Googleが開発したオープンソースで公開されている機械学習のライブラリ。PythonやC++などで記述することができる。現在のバージョンは2.2が最新になっている。Google Colabは2.2をデフォルトで設定されている。機械学習で有名なライブラリの一つ。
(注意 : バージョン1系と２系で動作が違う部分が多いので気を付ける。Qiitaなどの記事は1系がとても多い。)

2. **Keras**

Python言語で記述されたニューラルネットワークライブラリ。実際にはTensorflowなど上にいる高水準ライブラリという位置づけ。深層学習を簡単に実行できるようになった立役者。現在おそらく一番運用されている深層学習ツールだと思う。学術的に使うというよりは実装面で素早く作れることにたけているイメージ。
バージョンは2.3.1が最新で、Tensorflow２系からはTensorflow内部でのサポートに変わってtf.kerasというものの立ち位置に変わってきている。
(製作者はKeras自体のサポートはもう終わると発言。今後はtf.kerasがKerasとして機能。)

Keras系のライブラリの説明は大変なので省略。

3. **seaborn**

グラフ描画のためのライブラリ。有名なライブラリであるMatlabplotに並ぶ有名なライブラリ。

4. **Matlabplot**

グラフ描画のためのライブラリ。簡単にグラフを描画してくれる。

5. **numpy**

Python言語のための数値計算を効率的に行うためのライブラリ。計算を行うときにはより詳細的に、素早く計算をしてくれる。
多次元計算能力がかなり高くその点が高く評価されている。

6. PIL

画像を読み取るためにあるPythonの標準ライブラリ。RGB形式で読み取ってくれるためかなり直観的に画像を取り扱うことができる。画像ライブラリではOpencvも有名。どちらを使うかは結構好みがある。

---

## MNISTのデータを閲覧してみる

たくさんのライブラリがあることがわかってもらえたと思います。実際にはもう少し使う場合があります。
次に実際のMNISTのデータを見てみたいと思います。
MNISTはKerasだと関数を実行するだけですぐにロードすることができます。

In [ ]:
#Kerasの関数でデータの読み込み。データをシャッフルして学習データと検証データに分割してくれる
(x_train, y_train), (x_test, y_test) = mnist.load_data()
  
#5x5枚の画像を表示する
plt.figure(figsize=(10,10))
for i in range(25):
    rand_num=np.random.randint(0,50000)
    cifar_img=plt.subplot(5,5,i+1)
    plt.imshow(x_train[rand_num], cmap="gray")
    #x軸の目盛りを消す
    plt.tick_params(labelbottom='off')
    #y軸の目盛りを消す
    plt.tick_params(labelleft='off')
    #正解ラベルを表示
    plt.title(y_train[rand_num])


plt.show()

このような形でロードすることができます。ロードしたデータを一部表示してみました。

データのtrain, testというものはtrainが機械学習の勉強用のデータでtestが実際の試験として用いるデータになっています。

これは混ぜて学習することは絶対にダメです。人間でいうカンニングになってしまいます。(モデルの整合性チェックでやるときがあるけど基本ダメ。)

またx,yというのはxは画像データ、yはそれに対応する答えが格納されています。今回はxをみて学習してもらいyになるような勉強をしていくわけです。なので深層学習の中でも**教師あり学習**というものになります。

---

## データを整形する

表示した画像を見ると人間には簡単に数字として読み取ることができると思いますが、機械だとそうはいきません。なぜかというと数値の並びとしてしか機械は見ることができないからです。また数値というものを画像としてみたこともないから何がどうなのかもわかっていないと思います。

ロードした画像データは28*28の数字の並んだものになってます。
今回は読み込みやすいように784次元に引き伸ばします。
これをすることで機械に簡単に入れやすいようになります。
(今後はそうしません。)
次に型を画像データはfloat32,教師データはint8に固定します。軽量化や小数点計算などが理由です。

また教師データ(答え)は**Onehot形式**に変換していきます。
Onehotは一つだけがHIGHで他のデータがLOWになるようなデータであらわされるビット列を指します。
今回のデータは

```
教師データ(答え)
9
↓
Onehot形式の答え
state  : [0,1,2,3,4,5,6,7,8,9]
onehot : [0,0,0,0,0,0,0,0,0,1]
```

このような感じで９の状態のみをHIGHにするような形に教師データを加工します。これにすることで次元での計算に対応しやすくなります。

これで加工は完成します。


In [ ]:
# 学習用データの作成
# 読み込んだ手書き文字イメージデータを、機械学習の入力にできるよう整形します。
# クラス数の定義
num_classes = 10
# データをそれぞれ784次元に変換
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
# float型に変換する。
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 画像を正規化する。(計算速度の向上、精度の安定化)
x_train /= 255
x_test /= 255

# データ数の確認
print( 'Trainデータ数: ', x_train.shape[0])
print( 'Testデータ数: ',x_test.shape[0])

# 正解データをOnehot表記に変換

print('生データ(変更前)')
print(y_train[:5])

# intの32byte型にする。
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
# train,testそれぞれのデータをOnehot表記に変換する。
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test =  keras.utils.np_utils.to_categorical(y_test, num_classes)

print('Onehot表記に変換したもの')
print(y_train[:5])

---

次にニューラルネットワークの構造を定義していきます。
今回はCNNやRNNなどのようなものではなくかなりシンプルな構造のものを作成します。
Kerasはこのモデル定義をSequentialという定義の仕方とFunction APIと呼ばれる定義の仕方があります。Sequentialの方が直観的に定義することができるため今回はそちらを用いて定義します。

Sequentialモデルはaddを用いて様々な種類の層を付け足していくことで深層学習を実行できるような仕組みになっています。

今回は全結合層を３つ並べた単純なモデルで実験を行いたいと思います。
全結合層はKerasではDense,ChainerではLinearと呼ばれています。
入力に対して重みのフィルタ(隠れ層)を介して対応した次元数などに出力するような層になっています。
この層を利用して784次元を512次元にして一度フィルタを介して１０種類に仕分けるような構造になりました。
間のDropoutはフィルタの数をすべて使用するのではなく一部破棄して学習を続けるというものです。

実行するとモデルを生成してモデルの構造を文字と画像の２種類で出力されるようになってます。見てみましょう。

In [ ]:
# 簡単なモデルなのでSequentialモデルを使います
model = Sequential()


model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# モデルのコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# モデルアーキテクチャを画像に出力します
%matplotlib inline
plot_model(model, to_file='_.png', show_shapes=True, show_layer_names=True)
im = Image.open('_.png')
plt.figure(figsize=(10,10))
plt.axis('off')
_ = plt.imshow(np.array(im))

# モデルの出力結果を出します。
model.summary()


学習の設定を定義していきます。エポックやバッチサイズというものを設定していきます。
これは学習結果を更新する頻度を設定するものです。バッチサイズは学習データを指定数の個数のグループで分けていきそのグループを学習して学習時点で更新をかけていくものです。大体2の乗数で設定していくのがベターとされています。

```
例) 100個の学習データを10のバッチサイズに分けて学習
　　　　→　１度の学習で　100/10 = 10回　勉強を行う。　
```

エポックは反復回数といい、学習を何回繰り返すかを指定するものになってます。
難しい問題だと回数を増やしていきます。これが深層学習の学習が時間がかかるゆえんになってます。

model.fitで学習を開始します。

In [ ]:
# バッチサイズ
batch_size = 128

# 反復学習する回数
epochs = 10

# 学習の実行
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(x_test, y_test))

---

これで学習ができました。次に学習結果についてみていきましょう。
まずはデータ損失率や正解率を見ていきます。

In [ ]:
# 正答率と損失関数の値の推移をグラフ化します、
# 学習時の正答率などの情報は、history変数にすべて記録されています。

# 正答率の推移
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.xticks(np.arange(0, 10, 2))
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# 損失関数の推移
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.xticks(np.arange(0, 10, 2))
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

このようなグラフでデータ損失率や正解率を見ることができました。
これを見るとlossというものが下がっていてaccというものが上がっていくことがわかると思います。

lossは学習損失率といい、この深層学習で学習ができていない数値を表します。なのでこの数値を0にすることで学習が完了しているということになります。
またaccは正答率は教師データと照らし合わせてどれだけ正解しているのかを割合であらわすものになっています。なので100に近づくと完全に理解しているということになります。

またValidationというものがあると思います。これは検証データと呼ばれており人間の勉強での過程でいうと小テストの部分に当てはまります。
これがlossが低い、accが100に近い方がテストデータもうまく対応することができているという状態になっているといえます。

またtrainのデータがかなり高いけどvalidationデータが低い場合があります。これは過学習と呼ばれるトレーニングデータに過剰適合して検証データなどに適合しなくなってしまっている状態のことを言います。なんでこの過学習が起きないようにValidation dataなどを確認しながら学習を行っていきます。

---

次に実際にどのようにニューラルネットワークがデータを推論(Predict)しているのか確認していきます。

In [ ]:
# 表示するデータ数(頭から1～25まで対応)
display = 10

print("推論結果")
# 検証データに対する予測値の取得
y_predict = model.predict(x_test)
# 予測値を表示
print(y_predict[:display])
print("正解率")
# 正解データを表示
print(y_test[:display])

print("予測データ(整形後)")
# 予測した結果の中から一番確信度が高いデータを取り出す。(リストのインデックス)
predict_list = [np.argmax(prd) for prd in y_predict]
print(predict_list[:display])
print("正解データ(整形後)")
# 正解データの生データを表示する。(生データの変数を上でなくしたので復元する)
ans_list = [np.argmax(ans) for ans in y_test]
print(ans_list[:display])

# 推論を行ったMNISTデータの表示
fig = plt.figure(figsize=(9, 9))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
for i in range(display):
    ax = fig.add_subplot(5, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(x_test[i].reshape((28, 28)), cmap='gray')

このような形で手書き文字を判別することができました。
次はより複雑な画像データの推論をしてみましょう。
そのためにCNN(Convolution Nueral Network、畳み込みニューラルネットワークとも)を学習していきます。